In [1]:
from bs4 import BeautifulSoup
import requests




In [2]:
# Send a GET request to the URL
url = "https://www.regattanetwork.com/clubmgmt/applet_regatta_results.php?regatta_id=26198"
page = requests.get(url)

In [61]:
img_url = soup.body.img['src']
img_url = "https://www.regattanetwork.com/clubmgmt" + img_url[1:]
img_url


'https://www.regattanetwork.com/clubmgmt/club_uploads/204/SSS Logo.png'

In [65]:
import json
import pandas as pd
import requests
from dateutil import parser
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = "https://www.regattanetwork.com/clubmgmt/applet_regatta_results.php?regatta_id=26198"
page = requests.get(url)

# Initialize an empty dictionary to store JSON objects for each iteration
payload = {}

# Get the HTML content of the page
soup = BeautifulSoup(page.content, 'html.parser')

# Get title
title_text = soup.head.title.text

# Find event description
h4_tags = soup.find_all('h4')
event_description_tag = h4_tags[0].text.strip()

# Split the event description based on '\r\n'
event_description_parts = event_description_tag.split('\r\n')

# Split the second part of the event description based on ' | ' to get location and date
location_and_date = event_description_parts[1].split(' | ')

# Extract event name, location, and date
event_name = event_description_parts[0]
location = location_and_date[0]
date = location_and_date[1]

parsed_date = parser.parse(date)

# Download image
img_url = soup.body.img['src']
img_url = "https://www.regattanetwork.com/clubmgmt" + img_url[1:]
response = requests.get(img_url)

# Check if the request was successful
if response.status_code == 200:
    # Save the image to a file
    with open("event_image.jpg", "wb") as f:
        f.write(response.content)
else:
    print("Failed to download image")

# Add title, event name, location, date, and image URL to the payload dictionary
payload['title'] = title_text
payload['event_name'] = event_name
payload['location'] = location
payload['parsed_date'] = parsed_date.isoformat()
payload['image_url'] = img_url

# Find all tables containing race results
race_results = soup.find_all('div', class_='scoring-table-wrapper')

# Find all h2 tags containing fleet information
h2_tags = soup.find_all('h2')

# Iterate over each fleet and its corresponding race results
for index, tag in enumerate(h2_tags):
    # Extract fleet name and description
    fleet_name = tag.a['name']
    fleet_description = tag.find('font', size='3').text.strip()

    # Initialize the JSON object for this fleet
    fleet_data = {
        'fleet_name': fleet_name,
        'fleet_description': fleet_description
    }

    # Find the corresponding table for this fleet
    table = race_results[index].find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Read the HTML table into a DataFrame, skipping the first row
        df = pd.read_html(str(table), flavor='bs4', header=0)[0]
        
        # Drop specific columns if they exist
        columns_to_drop = ['Unnamed: 6', 'Unnamed: 11']
        df = df.drop(columns=columns_to_drop, errors='ignore')

        # Convert DataFrame to JSON-compatible format
        fleet_data['data'] = df.to_dict(orient='records')
    else:
        fleet_data['data'] = []  # Empty list if table does not exist

    # Add fleet data to the payload dictionary with index as key
    payload[index] = fleet_data

# Convert the payload dictionary to a JSON string
json_data = json.dumps(payload, indent=4)

# Now, json_data contains the JSON string representing the payload dictionary



C:\Users\danie\AppData\Local\Temp\ipykernel_29480\4166025702.py:81: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4', header=0)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\4166025702.py:81: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4', header=0)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\4166025702.py:81: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4', header=0)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\4166025702.py:81: FutureWarning: Passing literal html to 'read

In [66]:
print(json_data)

{
    "title": "\r\n2024 Cherry Pie Regatta SBYA Event - Series Standing",
    "event_name": "2024 Cherry Pie Regatta SBYA Event",
    "location": "  Sarasota Sailing Squadron",
    "parsed_date": "2024-02-17T00:00:00",
    "image_url": "https://www.regattanetwork.com/clubmgmt/club_uploads/204/SSS Logo.png",
    "0": {
        "fleet_name": "Melges15",
        "fleet_description": "Series Standing - 4 races scored",
        "data": [
            {
                "Pos": NaN,
                "Sail": NaN,
                "Boat": NaN,
                "Rating": NaN,
                "Skipper": NaN,
                "Yacht Club": NaN,
                "1": NaN,
                "2": NaN,
                "3": NaN,
                "4": NaN,
                "Total": NaN,
                "Pos.1": NaN
            },
            {
                "Pos": 1.0,
                "Sail": 107.0,
                "Boat": NaN,
                "Rating": 0.0,
                "Skipper": "Jeff Eiber",
            

In [48]:
import json
import pandas as pd

# Initialize an empty dictionary to store JSON objects for each iteration
payload = {}

# Assuming race_results contains the soup object with all the tables
for index, tag in enumerate(h2_tags):
    # Initialize the JSON object for this iteration
    json_object = {}

    # Extract fleet name and description
    fleet_name = tag.a['name']
    fleet_description = tag.find('font', size='3').text.strip()

    # Add fleet name and description to the JSON object
    json_object['fleet_name'] = fleet_name
    json_object['fleet_description'] = fleet_description

    # Find the table inside the corresponding race_results element
    table = race_results[index].find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Use Pandas to read the HTML table into a DataFrame, skipping the first row
        df = pd.read_html(str(table), flavor='bs4')[0]

        # Convert DataFrame to a list of dictionaries
        df_json = df.to_dict(orient='records')

        # Add DataFrame to the JSON object
        json_object['data'] = df_json
    else:
        json_object['data'] = []  # Empty list if table does not exist

    # Add the JSON object to the payload dictionary with index as key
    payload[index] = json_object

# Convert the payload dictionary to a JSON string
json_data = json.dumps(payload, indent=4)

# Now, json_data contains the JSON string representing the payload dictionary


C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\2110407307.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will 

In [49]:
df

,Pos,Sail,Boat,Rating,Skipper,Yacht Club,Corrected,Unnamed: 7,1,Unnamed: 9,Total,Pos.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,777.0,Whole Lotta Rosie,18.0,Jon Dowd,Sarasota Sailing Squadron,00:47:30,NaN,1,NaN,1.0,1.0
2,2.0,7.0,Sunspot,9.0,Bill Johnsen,William Johnsen,00:49:05,NaN,2,NaN,2.0,2.0
3,3.0,12.0,Triple Threat,-33.0,John Novak,Sarasota Sailing Squadron,00:50:38,NaN,3,NaN,3.0,3.0
4,4.0,187.0,Krakian,24.0,Alan Staniforth,SSS/SYS,00:52:58,NaN,4,NaN,4.0,4.0
5,5.0,138.0,NaN,-15.0,NaN,Sss,NO TIME,NaN,6/DNC,NaN,6.0,5.0


In [29]:
import pandas as pd

# Initialize an empty list to store DataFrames for each table
dfs = []

# Assuming race_results contains the soup object with all the tables
for table_wrapper in race_results:
    # Find the table inside the wrapper
    table = table_wrapper.find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Use Pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table), flavor='bs4')[0]
        
        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print("No table found in the wrapper.")

# Now, dfs contains DataFrames for each table
# You can access each DataFrame using index, e.g., dfs[0], dfs[1], etc.


C:\Users\danie\AppData\Local\Temp\ipykernel_29480\851695625.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\851695625.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\851695625.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), flavor='bs4')[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_29480\851695625.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be r

In [10]:
import json
from bs4 import BeautifulSoup

# Initialize a list to store JSON objects for each table
all_table_data = []


race_results = soup.find_all('div', class_='scoring-table-wrapper')

# Assuming race_results contains the soup object with all the tables
for table_wrapper in race_results:
    # Find the table inside the wrapper
    table = table_wrapper.find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Initialize a list to store JSON data for this table
        table_data = []

        # Iterate through each row in the table, skipping the first row which contains the headers
        for row in table.find_all('tr')[1:]:
            # Extract data from each column in the row
            columns = row.find_all('td')
            position = columns[0].text.strip()
            sail_number = columns[1].text.strip()
            skipper_name = columns[4].text.strip()
            total_score = columns[-2].text.strip()

            # Create a JSON object for this row and append it to the table data list
            row_data = {
                'Position': position,
                'Sail Number': sail_number,
                'Skipper Name': skipper_name,
                'Total Score': total_score
            }
            table_data.append(row_data)

        # Append the JSON data for this table to the list containing all tables' data
        all_table_data.append(table_data)
    else:
        print("No table found in the wrapper.")

# Convert the list of table data JSON objects to a JSON string
json_data = json.dumps(all_table_data, indent=4)

# Print the JSON string
print(json_data)


[
    [
        {
            "Position": "Pos",
            "Sail Number": "Sail",
            "Skipper Name": "Skipper",
            "Total Score": "Total"
        },
        {
            "Position": "1",
            "Sail Number": "107",
            "Skipper Name": "Jeff Eiber",
            "Total Score": "4"
        },
        {
            "Position": "2",
            "Sail Number": "107",
            "Skipper Name": "",
            "Total Score": "8"
        },
        {
            "Position": "3",
            "Sail Number": "011",
            "Skipper Name": "Eva McKee",
            "Total Score": "13"
        },
        {
            "Position": "4",
            "Sail Number": "013",
            "Skipper Name": "LANCE THOMAS",
            "Total Score": "15"
        },
        {
            "Position": "5",
            "Sail Number": "1",
            "Skipper Name": "Jeff Nicholas",
            "Total Score": "21"
        },
        {
            "Position": "6",
            

In [23]:
import pandas as pd

# Initialize an empty list to store DataFrames for each table
dfs = []

# Assuming race_results contains the soup object with all the tables
for table_wrapper in race_results:
    # Find the table inside the wrapper
    table = table_wrapper.find('table', class_='scoring-table')
    
    # Check if table exists
    if table:
        # Extract column headers
        headers = [header.text.strip() for header in table.find_all('td')]
        
        # Initialize a list to store data for this table
        data = []

        # Iterate through each row in the table, skipping the first row which contains the headers
        for row in table.find_all('tr')[1:]:
            # Extract data from each column in the row
            columns = row.find_all('td')
            row_data = [column.text.strip() for column in columns]

            # Append the data for this row to the table data list
            data.append(row_data)

        # Create a DataFrame for this table
        df = pd.DataFrame(data, columns=headers)
        
        # Append the DataFrame to the list of DataFrames
        dfs.append(df)
    else:
        print("No table found in the wrapper.")

# Now, dfs will contain a list of DataFrames, each representing a table from the HTML


ValueError: 126 columns passed, passed data had 14 columns

In [185]:
h4_tags = soup.find_all('h4')
event_description_tag = h4_tags[0].text.strip()

# Split the event description based on '\r\n'
event_description_parts = event_description_tag.split('\r\n')

# Split the second part of the event description based on ' | ' to get location and date
location_and_date = event_description_parts[1].split(' | ')

# Extract event name, location, and date
event_name = event_description_parts[0]
location = location_and_date[0]
date = location_and_date[1]

print("Event Name:", event_name)
print("Location:", location)
print("Date:", date)


Event Name: 2024 Cherry Pie Regatta SBYA Event
Location:   Sarasota Sailing Squadron
Date:         February 17, 2024


In [161]:
h2_tags = soup.find_all('h4')
for tag in h2_tags:
    print(tag)

<h4>2024 Cherry Pie Regatta SBYA Event<br/>
  Sarasota Sailing Squadron |         February 17, 2024        </h4>
<h4>
</h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 4:07:38 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 4:18:07 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturday, February 17, 2024 3:55:38 PM CDT<br/>Click on race number to view detailed race information.</small></font></h4>
<h4><font size="2">
        Information is provisional and subject to modification<br/>
<small>Regatta results last updated: Saturda

In [48]:
h2 = soup.find('h2')
print(h2)


<h2>
<a href="/clubmgmt/applet_regatta_results.php?regatta_id=26198&amp;limit_fleet=Melges15" name="Melges15"><font size="4">Melges15</font></a> <font size="4"><small><small>(<small>8</small> boats) <a href="#top">(top)<br/>
</a></small></small></font><font size="3">
  Series Standing - 4 races scored</font></h2>


In [36]:
soup

results_tables = soup.find_all('table', class_ = 'responsive scoring-table')



for tables in results_tables:
    print(tables.title)

None
None
None
None
None
None
None
None
None
None


In [27]:
soup.find_all('table')[3
]

<table bgcolor="#999999" class="responsive scoring-table">
<tr>
<!-- <td bgcolor="#999999"> -->
<thead>
<tr>
<td align="center" bgcolor="#6699CC" class="pos" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Pos</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Sail  </b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Boat  </b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Rating</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Skipper</b></font></font></td>
<td bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>Yacht Club</b></font></font></td>
<td align="center" bgcolor="#999999" valign="bottom"></td>
<td align="center" bgcolor="#6699CC" valign="bottom"><font size="2"><font face="Verdana" size="2"><b>  <a h

In [18]:
soup = BeautifulSoup(page.text, 'html.parser')

In [20]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <meta content="600" http-equiv="Refresh"/>
  <title>
   2024 Cherry Pie Regatta SBYA Event - Series Standing
  </title>
  <link href="mgmt_style.css" rel="stylesheet" type="text/css"/>
  <meta content="width=device-width, initial-scale=1.0" id="view" name="viewport">
   <link href="/html/css/global.min.css" rel="stylesheet" type="text/css"/>
   <script src="/js/jquery.js">
   </script>
   <script src="/js/jquerymigrate.js">
   </script>
   <script src="/html/js/global.min.js?v=202002241401" type="text/javascript">
   </script>
   <meta content="https://www.regattanetwork.com/images/USRN_icon200.png" property="og:image">
   </meta>
  </meta>
 </head>
 <!-- <link rel="stylesheet" href="./css/responsive-tables.css"> -->
 <script href="/clubmgmt/js/respo